In [1]:
import forecastio
import datetime
import pandas as pd
import pickle

In [2]:
def get_weather(start_date, end_date, lat, lon, api_key):
    
    year_raw = start_date[0]
    month_raw = start_date[1]
    day_raw = start_date[2]
    
    weather_data = []
    
    while [year_raw, month_raw, day_raw]!= end_date:
        
        year = str(year_raw)
        month = "%02d" % (month_raw)
        day = "%02d" % (day_raw)

        url = "https://api.forecast.io/forecast/"+api_key+"/"+lat+","+lon+","+year+"-"+month+"-"+day+"T12:00:00"
                        
        forecast = forecastio.manual(url)
        
        weather_data.append(forecast.json)
        
        if (month_raw in [1,3,5,7,8,10] and day_raw == 31):
            day_raw = 1
            month_raw += 1
        
        elif (month_raw in [4, 6, 9, 11] and day_raw == 30):
            day_raw = 1
            month_raw += 1
            
        elif (month_raw == 2 and day_raw ==28 and year_raw%4 == 0):
            day_raw += 1
            
        elif (month_raw == 2 and day_raw == 29):
            day_raw = 1
            month_raw = 3            
        
        elif (month_raw == 12 and day_raw == 31):
            year_raw += 1
            month_raw = 1
            day_raw = 1
            
        else:
            day_raw += 1
            
    return weather_data

In [3]:
def parse_weather_data(weather_file):
    
    parsed_data = {}
    
    for i in range(len(weather_file)):
        unixtime = weather_file[i]['daily']['data'][0]['time']
        
        temperatureMin = weather_file[i]['daily']['data'][0]['temperatureMin']
        temperatureMax = weather_file[i]['daily']['data'][0]['temperatureMax']
        avgTemperature = (temperatureMin + temperatureMax)/2
        windSpeed = weather_file[i]['daily']['data'][0]['windSpeed']
        precipProbability = weather_file[i]['daily']['data'][0]['precipProbability']
        precipIntensity = weather_file[i]['daily']['data'][0]['precipIntensity']
        
        parsed_data[unixtime] = [avgTemperature, windSpeed, precipProbability, precipIntensity]
        
    return parsed_data        

In [15]:
def parse_weather_data_alt(weather_file):
    
    parsed_data = {}
    
    for i in range(len(weather_file)):
        unixtime = weather_file[i]['daily']['data'][0]['time']
        
        temperatureMin = weather_file[i]['daily']['data'][0]['temperatureMin']
        temperatureMax = weather_file[i]['daily']['data'][0]['temperatureMax']
        avgTemperature = (temperatureMin + temperatureMax)/2
        windSpeed = weather_file[i]['daily']['data'][0]['windSpeed']
        precipType = weather_file[i]['daily']['data'][0]['precipType']
        
        parsed_data[unixtime] = [avgTemperature, windSpeed, precipType]
        
    return parsed_data        

In [4]:
api_key = "8a8bd95f58ebd66ceda140c4cae56f91"

nyc_lat = "40.748817"
nyc_lon = "-73.985428"

tokyo_lat = "35.685360"
tokyo_lon = "139.753372"

nyc_weather = {}
tokyo_weather = {}

start_date = [2012,4,12]
end_date = [2013,2,16]

In [5]:
nyc_weather = get_weather(start_date, end_date, nyc_lat, nyc_lon, api_key)

In [11]:
nyc_weather[0]['daily']

{u'data': [{u'apparentTemperatureMax': 58.13,
   u'apparentTemperatureMaxTime': 1334260800,
   u'apparentTemperatureMin': 38.11,
   u'apparentTemperatureMinTime': 1334228400,
   u'cloudCover': 0.62,
   u'dewPoint': 32.54,
   u'humidity': 0.52,
   u'icon': u'partly-cloudy-day',
   u'moonPhase': 0.72,
   u'precipIntensity': 0,
   u'precipIntensityMax': 0,
   u'precipProbability': 0,
   u'pressure': 1014.94,
   u'summary': u'Mostly cloudy throughout the day.',
   u'sunriseTime': 1334226121,
   u'sunsetTime': 1334273597,
   u'temperatureMax': 58.13,
   u'temperatureMaxTime': 1334260800,
   u'temperatureMin': 43.55,
   u'temperatureMinTime': 1334224800,
   u'time': 1334203200,
   u'visibility': 10,
   u'windBearing': 344,
   u'windSpeed': 10.42}]}

In [6]:
parsed_nyc_weather = parse_weather_data(nyc_weather)

In [7]:
pickle.dump(parsed_nyc_weather, open("parsed_nyc_weather.pickle", "wb"))

In [8]:
tokyo_weather = get_weather(start_date, end_date, tokyo_lat, tokyo_lon, api_key)

In [10]:
tokyo_weather[0]['daily']

{u'data': [{u'apparentTemperatureMax': 71.57,
   u'apparentTemperatureMaxTime': 1334203200,
   u'apparentTemperatureMin': 52.92,
   u'apparentTemperatureMinTime': 1334170800,
   u'cloudCover': 0.43,
   u'dewPoint': 46.72,
   u'humidity': 0.59,
   u'icon': u'partly-cloudy-day',
   u'moonPhase': 0.7,
   u'precipType': u'rain',
   u'pressure': 1003.13,
   u'summary': u'Partly cloudy throughout the day.',
   u'sunriseTime': 1334175202,
   u'sunsetTime': 1334221938,
   u'temperatureMax': 71.57,
   u'temperatureMaxTime': 1334203200,
   u'temperatureMin': 52.92,
   u'temperatureMinTime': 1334170800,
   u'time': 1334156400,
   u'visibility': 7.06,
   u'windBearing': 230,
   u'windSpeed': 1.82}]}

In [16]:
parsed_tokyo_weather = parse_weather_data_alt(tokyo_weather)

In [17]:
pickle.dump(parsed_tokyo_weather, open("parsed_tokyo_weather.pickle", "wb"))